In [3]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from nltk.tokenize import regexp_tokenize

In [4]:
Train = pd.read_csv('Movies\Train.csv')
Test = pd.read_csv('Movies\Test.csv')

## *TEst Model

In [5]:
#cv = CountVectorizer(min_df=0.4,tokenizer=)

In [6]:
def reg_exp_toke(text):
    tokenized_text = regexp_tokenize(text,pattern='[^\d\W]+')
    return tokenized_text

In [7]:
cv = CountVectorizer(min_df=0.00084,tokenizer=reg_exp_toke)

In [8]:
cv.fit(Train.review)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=0.00084,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function reg_exp_toke at 0x000001921B839948>,
                vocabulary=None)

In [9]:
print(len(cv.vocabulary_))

11257


In [10]:
Test.shape

(10000, 1)

In [11]:
cv.vocabulary_.get('wholesome')
Train.head()

,review,label
0,mature intelligent and highly charged melodram...,pos
1,http://video.google.com/videoplay?docid=211772...,pos
2,Title: Opera (1987) Director: Dario Argento Ca...,pos
3,I think a lot of people just wrote this off as...,pos
4,This is a story of two dogs and a cat looking ...,pos


In [12]:
for i in range(Train.shape[0]):
    Train.review[i] = reg_exp_toke(Train.review[i])
    #Test.review[i] = reg_exp_toke(Test.review[i])
Train.head()

for i in range(Test.shape[0]):
    Test.review[i] = reg_exp_toke(Test.review[i])
Test.head()

,review
0,"[Remember, those, old, kung, fu, movies, we, u..."
1,"[This, movie, is, another, one, on, my, List, ..."
2,"[How, in, the, world, does, a, thing, like, th..."
3,"[Queen, of, the, Damned, is, one, of, the, bes..."
4,"[The, Caprica, episode, S, E, is, well, done, ..."


In [13]:
for i in range(Train.shape[0]):
    for w in Train.review[i]:
        if w not in cv.vocabulary_:
            Train.review[i].remove(w)
            #Test.review[i].remove(w)
            
for i in range(Test.shape[0]):
    for w in Test.review[i]:
        if w not in cv.vocabulary_:
            Test.review[i].remove(w)

In [14]:
Train.head()

,review,label
0,"[mature, intelligent, and, highly, charged, me...",pos
1,"[http, video, com, docid, en, was, tried, br, ...",pos
2,"[Dario, Cast, Masillach, Charleson, Barberini,...",pos
3,"[think, a, lot, of, people, just, wrote, this,...",pos
4,"[is, a, story, of, two, dogs, and, a, cat, loo...",pos


In [15]:
rev_voc = dict([value,key] for (key,value) in cv.vocabulary_.items())

In [16]:
#print(rev_voc)
#print('*************************************************************')
#print(cv.vocabulary_)

for i in range(Train.shape[0]):
    Train.review[i] = [cv.vocabulary_.get(item,item) for item in Train.review[i]]
    
for i in range(Test.shape[0]):
    Test.review[i] = [cv.vocabulary_.get(item,item) for item in Test.review[i]]
    

In [17]:
print(Train.head())
print(Test.head())

                                              review label
0  [6191, 5221, 365, 4725, 1577, 6253, 3817, 5045...   pos
1  [4876, 10758, 1877, docid, 3299, 10911, 10403,...   pos
2  [Dario, Cast, Masillach, Charleson, Barberini,...   pos
3  [10129, 0, 5958, 6896, 7266, 5493, 11191, 1014...   pos
4  [5331, 0, 9627, 6896, 10474, 2937, 365, 0, 148...   pos
                                              review
0  [10146, 6924, 5620, 4091, 6542, 10942, 10660, ...
1  [6540, 5331, 411, 6940, 6938, 6595, 6896, Not,...
2  [5045, 10091, 11147, 2933, 0, 10127, 5827, 101...
3  [6896, 10091, 5331, 6940, 6896, 10091, 930, 10...
4  [3391, E, 5331, 10978, 2957, 560, 0, 7375, 101...


In [18]:
for i in range(10):
    for i in range(Train.shape[0]):
        for w in Train.review[i]:
            if type(w) == str:
                Train.review[i].remove(w)
                
for i in range(10):
    for i in range(Test.shape[0]):
        for w in Test.review[i]:
            if type(w) == str:
                Test.review[i].remove(w)

In [19]:
Train.head()

,review,label
0,"[6191, 5221, 365, 4725, 1577, 6253, 3817, 5045...",pos
1,"[4876, 10758, 1877, 3299, 10911, 10403, 1143, ...",pos
2,"[6945, 7014, 6540, 4504, 8831, 10911, 365, 100...",pos
3,"[10129, 0, 5958, 6896, 7266, 5493, 11191, 1014...",pos
4,"[5331, 0, 9627, 6896, 10474, 2937, 365, 0, 148...",pos


In [20]:
le = LabelEncoder()
Train['label'] = le.fit_transform(Train['label'])
Train.head()

,review,label
0,"[6191, 5221, 365, 4725, 1577, 6253, 3817, 5045...",1
1,"[4876, 10758, 1877, 3299, 10911, 10403, 1143, ...",1
2,"[6945, 7014, 6540, 4504, 8831, 10911, 365, 100...",1
3,"[10129, 0, 5958, 6896, 7266, 5493, 11191, 1014...",1
4,"[5331, 0, 9627, 6896, 10474, 2937, 365, 0, 148...",1


In [21]:
from keras.preprocessing import sequence
X_Train = sequence.pad_sequences(Train.review,maxlen=500)
X_Test = sequence.pad_sequences(Test.review,maxlen=500)

Using TensorFlow backend.
c:\users\vj\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\vj\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\vj\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:

In [22]:
print(X_Train.shape)

print(X_Test.shape)

(40000, 500)
(10000, 500)


In [23]:
from keras.models import Sequential
from keras.layers import Embedding,SimpleRNN,Dense

In [24]:
model = Sequential()
model.add(Embedding(11257,64))
model.add(SimpleRNN(32))
model.add(Dense(1,activation='sigmoid'))
model.summary()




Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 64)          720448    
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 32)                3104      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 723,585
Trainable params: 723,585
Non-trainable params: 0
_________________________________________________________________


In [25]:
model.compile(optimizer='adam',metrics=['accuracy'],loss='binary_crossentropy')



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [26]:
from keras.callbacks import ModelCheckpoint,EarlyStopping
#checkpoint = ModelCheckpoint('best_RNN_model', monitor='val_loss',verbose=0,save_best_only=True, save_weights_only=False, mode='auto', period=1)

model.fit(X_Train,Train.label,validation_split=0.2,epochs = 5,batch_size=128)


Train on 32000 samples, validate on 8000 samples
Epoch 1/5
32000/32000 [==============================] - 32s 1ms/step - loss: 0.6120 - acc: 0.6464 - val_loss: 0.4631 - val_acc: 0.7924
Epoch 2/5
32000/32000 [==============================] - 32s 988us/step - loss: 0.3769 - acc: 0.8393 - val_loss: 0.3782 - val_acc: 0.8429
Epoch 3/5
32000/32000 [==============================] - 32s 991us/step - loss: 0.2791 - acc: 0.8893 - val_loss: 0.3472 - val_acc: 0.8620
Epoch 4/5
32000/32000 [==============================] - 32s 990us/step - loss: 0.2381 - acc: 0.9044 - val_loss: 0.6059 - val_acc: 0.6701
Epoch 5/5
32000/32000 [==============================] - 32s 1ms/step - loss: 0.2702 - acc: 0.8858 - val_loss: 0.5642 - val_acc: 0.7758


In [27]:
YT = model.predict(X_Test)

In [28]:
for i in range(YT.shape[0]):
    YT[i] = np.round_(YT[i])

In [29]:
for i in range(10):
    print(YT[i])

[0.]
[0.]
[0.]
[1.]
[1.]
[0.]
[0.]
[0.]
[0.]
[1.]


In [30]:
"""for i in range(YT.shape[0]):
    if YT[i] == 1:
        YT[i] = 'pos'
    elif YT[i] == 0:
        YT[i] = 'neg'"""

#YT = le.inverse_transform(YT)

for i in range(YT.shape[0]):
    YT[i] = np.int(YT[i])

In [37]:
#YT = le.inverse_transform(YT)
YT1 = []

for i in range(YT.shape[0]):
    if YT[i] == 0:
        YT1.append('neg')
    elif YT[i] ==1:
        YT1.append('pos')

In [43]:
#YT = le.inverse_transform(YT)

In [42]:
YT1 = pd.DataFrame(YT1)
YT1.to_csv('YTEST_MOVIES',index = True)

In [41]:
print(YT[56])
print(YT1[56])

[1.]
pos


In [33]:
#import numpy as np